In [199]:
import pandas as pd
import datetime
import os
import numpy as np

In [200]:
week_list=['26','27','28','29','30','31']

In [201]:
Krogger_2017_128=pd.read_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/data/Kroger - Tangy 128oz Only - FY 2017 - FY 2018 YTD w.e. Aug 3 - Weekly Store w_ Kiosk.csv",dtype=str,skiprows=8)
Krogger_2017_64=pd.read_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/data/Kroger - Tangy 64oz Only - FY 2017 - FY 2018 YTD w.e. Aug 11 - Weekly Store w_ Kiosk.csv",dtype=str,skiprows=8)

Krogger_2018=pd.read_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/data/Kroger - Tangy 128oz and 64oz FY 2018 YTD w.e. Sep 8 - Weekly Store w_ Kiosk.csv",dtype=str,skiprows=8)


In [202]:
def clean_df(df):
    df=df[['WEEK_NAME','UPC','SCANNED_RETAIL_DOLLARS','SCANNED_MOVEMENT','STO_NAME','STO_ZIP','STO_ADDRESS','ITM_SCN_PRC_GRP_DESC']]
    
    df['year']=df['WEEK_NAME'].apply(lambda x: x[:4])
    
    df['week']=np.where(df['year']=='2017',df['WEEK_NAME'].apply(lambda x: str(int(x[len(x)-3:len(x)-1])-1)),
                        df['WEEK_NAME'].apply(lambda x: x[len(x)-3:len(x)-1]))
    df['zip_cd']=df['STO_ZIP'].apply(lambda x: x[:5])
    df=df[df['week'].isin(week_list)]
    df=df.rename(columns={"SCANNED_RETAIL_DOLLARS":"sales","SCANNED_MOVEMENT":"units","STO_NAME":"store",
                          'STO_ADDRESS':'address',"ITM_SCN_PRC_GRP_DESC":"product"})
    df['sales']=df['sales'].astype(float)
    df['units']=df['units'].astype(float)
    return df[['store','year','week','zip_cd','address','sales','units','product']]

In [203]:
Krogger_2017_128=clean_df(Krogger_2017_128)
Krogger_2017_64=clean_df(Krogger_2017_64)
Krogger_2018=clean_df(Krogger_2018)
Krogger_2017_128=Krogger_2017_128[Krogger_2017_128['year']=='2017']
Krogger_2017_64=Krogger_2017_64[Krogger_2017_64['year']=='2017']
Krogger_2018=Krogger_2018[Krogger_2018['year']=='2018']


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [204]:
Zip_IRI_mapping=pd.read_excel("/home/jian/SunnyD/Otherinput/All_IRI_Mkt_with_Zips_0430.xlsx",dtype=str)

In [205]:
def func(x):
    y=len(list(set(x)))
    return y

Krogger_both_year=Krogger_2018.append(Krogger_2017_128).append(Krogger_2017_64)
Krogger_both_year['store_info']=Krogger_both_year['store']+"_"+Krogger_both_year['zip_cd']+"_"+Krogger_both_year['address']
Krogger_both_year['year_week']=Krogger_both_year['year']+"_"+Krogger_both_year['week']
sales_available_stores=Krogger_both_year.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index().rename(columns={"year_week":"weeks_with_sales"})
writer=pd.ExcelWriter("/home/jian/SunnyD/On_Going_WM_KG/Krogger/output/Investigate_in_Krogger_RALEIGH_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
sales_available_stores.to_excel(writer,"summary_count_weeks",index=False)
writer.save()


# Combined Comparison

In [206]:
Krogger_both_year=Krogger_2018.append(Krogger_2017_128).append(Krogger_2017_64)
Krogger_both_year['store_info']=Krogger_both_year['store']+"_"+Krogger_both_year['zip_cd']+"_"+Krogger_both_year['address']
Krogger_both_year['year_week']=Krogger_both_year['year']+"_"+Krogger_both_year['week']
Krogger_both_year=Krogger_both_year[Krogger_both_year['sales']>0]
Krogger_both_year.shape

(54584, 10)

In [207]:
len(Krogger_both_year['store_info'].unique())

2414

In [208]:
def func(x):
    y=len(list(set(x)))
    return y

In [209]:
Krogger_both_year_existing=Krogger_both_year.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index()
Krogger_both_year_existing=Krogger_both_year_existing[Krogger_both_year_existing['year_week']==12]['store_info'].unique().tolist()
Krogger_both_year=Krogger_both_year[Krogger_both_year['store_info'].isin(Krogger_both_year_existing)]



In [210]:
Krogger_both_year_compare=Krogger_both_year[Krogger_both_year['week'].isin(week_list)]
Krogger_both_year_compare=pd.merge(Krogger_both_year_compare,Zip_IRI_mapping,on='zip_cd',how="left")
Krogger_both_year_compare.shape

(52930, 11)

In [211]:
Krogger_both_year_compare[(Krogger_both_year_compare['year']=='2017')]['week'].unique()

array(['26', '27', '28', '29', '30', '31'], dtype=object)

In [212]:
Markets_18=['BIRMINGHAM/MONTGOMERY','NEW ORLEANS/MOBILE','PHOENIX/TUCSON','LOS ANGELES','DENVER','MIAMI/FT LAUDERDALE',
            'ATLANTA','SOUTH CAROLINA','CHICAGO','BOSTON','BALTIMORE/WASHINGTON','DETROIT','MISSISSIPPI','RALEIGH/GREENSBORO',
            'NEW YORK','WEST TEX/NEW MEXICO','HOUSTON','DALLAS/FT WORTH']

def In_18(x):
    if x in Markets_18:
        y="In_18_Mkts"
    else:
        y="Other"
    return y
Krogger_both_year_compare['18_Mkt_Ind']=Krogger_both_year_compare['IRI Market'].apply(lambda x: In_18(x))

In [213]:
df_YoY_compare_summary=Krogger_both_year_compare.groupby(['18_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()
df_YoY_compare_summary.to_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/output/SunnyD_Krogger_4_weeks_YoY_"+str(datetime.datetime.now().date())+".csv",index=False)

In [214]:
df_YoY_compare_summary

,18_Mkt_Ind,year,week,sales,units
0,In_18_Mkts,2017,26,115182.99,54656.0
1,In_18_Mkts,2017,27,100696.11,45765.0
2,In_18_Mkts,2017,28,83352.41,38547.0
3,In_18_Mkts,2017,29,85881.25,40766.0
4,In_18_Mkts,2017,30,82996.07,38915.0
5,In_18_Mkts,2017,31,86800.69,40565.0
6,In_18_Mkts,2018,26,91114.30,48641.0
7,In_18_Mkts,2018,27,84121.49,36877.0
8,In_18_Mkts,2018,28,75058.49,30736.0
9,In_18_Mkts,2018,29,69608.84,27599.0


# 64, 128 respectively comparison

In [215]:
Krogger_both_year_64=Krogger_both_year[Krogger_both_year['product']=="Sunny D 64 FO"]
Krogger_both_year_existing_64=Krogger_both_year_64.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index()
Krogger_both_year_existing_64=Krogger_both_year_existing_64[Krogger_both_year_existing_64['year_week']==12]['store_info'].unique().tolist()
Krogger_both_year_64=Krogger_both_year_64[Krogger_both_year_64['store_info'].isin(Krogger_both_year_existing_64)]
Krogger_both_year_compare_64=Krogger_both_year_64[Krogger_both_year_64['week'].isin(week_list)]
Krogger_both_year_compare_64=pd.merge(Krogger_both_year_compare_64,Zip_IRI_mapping,on='zip_cd',how="left")
Krogger_both_year_compare_64['18_Mkt_Ind']=Krogger_both_year_compare_64['IRI Market'].apply(lambda x: In_18(x))
df_YoY_compare_summary_64=Krogger_both_year_compare_64.groupby(['18_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()


In [216]:
Krogger_both_year_128=Krogger_both_year[Krogger_both_year['product']=="Sunny D 128 FO"]
Krogger_both_year_existing_128=Krogger_both_year_128.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index()
Krogger_both_year_existing_128=Krogger_both_year_existing_128[Krogger_both_year_existing_128['year_week']==12]['store_info'].unique().tolist()
Krogger_both_year_128=Krogger_both_year_128[Krogger_both_year_128['store_info'].isin(Krogger_both_year_existing_128)]
Krogger_both_year_compare_128=Krogger_both_year_128[Krogger_both_year_128['week'].isin(week_list)]
Krogger_both_year_compare_128=pd.merge(Krogger_both_year_compare_128,Zip_IRI_mapping,on='zip_cd',how="left")
Krogger_both_year_compare_128['18_Mkt_Ind']=Krogger_both_year_compare_128['IRI Market'].apply(lambda x: In_18(x))
df_YoY_compare_summary_128=Krogger_both_year_compare_128.groupby(['18_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()


In [217]:
Krogger_both_year_compare_2_product=Krogger_both_year[(Krogger_both_year['store_info'].isin(Krogger_both_year_existing_64)) &\
                                                     (Krogger_both_year['store_info'].isin(Krogger_both_year_existing_128))]
Krogger_both_year_compare_2_product=Krogger_both_year_compare_2_product[Krogger_both_year_compare_2_product['week'].isin(week_list)]
Krogger_both_year_compare_2_product=pd.merge(Krogger_both_year_compare_2_product,Zip_IRI_mapping,on='zip_cd',how="left")
Krogger_both_year_compare_2_product['18_Mkt_Ind']=Krogger_both_year_compare_2_product['IRI Market'].apply(lambda x: In_18(x))
df_YoY_compare_summary_2_product=Krogger_both_year_compare_2_product.groupby(['18_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()


In [218]:
Krogger_both_year_compare_2_product.shape

(46586, 12)

In [219]:
writer=pd.ExcelWriter("/home/jian/SunnyD/On_Going_WM_KG/Krogger/output/SunnyD_Krogger_4_weeks_YoY_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

In [220]:
def YoY_summary_df(df):
    df_2017_sales=df[df['year']=='2017'].rename(columns={"sales":"sales_2017"})
    df_2018_sales=df[df['year']=='2018'].rename(columns={"sales":"sales_2018"})
    del df_2017_sales['units']
    del df_2018_sales['units']
    del df_2017_sales['year']
    del df_2018_sales['year']
    sales_yoy=pd.merge(df_2017_sales,df_2018_sales,on=["18_Mkt_Ind",'week'],how="left")
    
    df_2017_units=df[df['year']=='2017'].rename(columns={"units":"units_2017"})
    df_2018_units=df[df['year']=='2018'].rename(columns={"units":"units_2018"})
    del df_2017_units['sales']
    del df_2018_units['sales']
    del df_2017_units['year']
    del df_2018_units['year']
    units_yoy=pd.merge(df_2017_units,df_2018_units,on=["18_Mkt_Ind",'week'],how="left")
    
    output=pd.merge(sales_yoy,units_yoy,on=["18_Mkt_Ind",'week'],how="left")
    output['sales_yoy']=(output['sales_2018']-output['sales_2017'])/output['sales_2017']
    output['units_yoy']=(output['units_2018']-output['units_2017'])/output['units_2017']
    return output

In [221]:
df_YoY_compare_summary_128

,18_Mkt_Ind,year,week,sales,units
0,In_18_Mkts,2017,26,80782.09,34662.0
1,In_18_Mkts,2017,27,66896.67,25996.0
2,In_18_Mkts,2017,28,50399.73,18382.0
3,In_18_Mkts,2017,29,52772.06,20818.0
4,In_18_Mkts,2017,30,52686.99,21216.0
5,In_18_Mkts,2017,31,54497.99,21739.0
6,In_18_Mkts,2018,26,48246.51,15287.0
7,In_18_Mkts,2018,27,54526.45,17951.0
8,In_18_Mkts,2018,28,50423.45,17057.0
9,In_18_Mkts,2018,29,47152.21,15371.0


In [222]:
summary_64=YoY_summary_df(df_YoY_compare_summary_64)
summary_128=YoY_summary_df(df_YoY_compare_summary_128)
summary_both_respective=YoY_summary_df(df_YoY_compare_summary_2_product)
summary_both_together=YoY_summary_df(df_YoY_compare_summary)

In [223]:
summary_64.to_excel(writer,"summary_64",index=False)
summary_128.to_excel(writer,"summary_128",index=False)
summary_both_respective.to_excel(writer,"summary_both_respective",index=False)
summary_both_together.to_excel(writer,"summary_both_together",index=False)

In [224]:
writer.save()

In [225]:
summary_both_respective

,18_Mkt_Ind,week,sales_2017,sales_2018,units_2017,units_2018,sales_yoy,units_yoy
0,In_18_Mkts,26,108492.75,86136.95,51477.0,46303.0,-0.206058,-0.100511
1,In_18_Mkts,27,94680.21,79240.15,42846.0,34705.0,-0.163076,-0.190006
2,In_18_Mkts,28,77835.99,70469.69,35618.0,28675.0,-0.094639,-0.194930
3,In_18_Mkts,29,80165.80,65551.20,37841.0,25867.0,-0.182305,-0.316429
4,In_18_Mkts,30,78137.03,60935.71,36506.0,23419.0,-0.220143,-0.358489
5,In_18_Mkts,31,81457.25,63470.71,37996.0,24895.0,-0.220810,-0.344799
6,Other,26,104604.48,86526.15,48702.0,36559.0,-0.172826,-0.249333
7,Other,27,97277.55,90180.60,43708.0,35897.0,-0.072956,-0.178709
8,Other,28,99791.75,86628.20,51112.0,33786.0,-0.131910,-0.338981
9,Other,29,101489.48,77964.78,55338.0,29703.0,-0.231794,-0.463244
